In [30]:
import os
import shutil
import pickle 
from typing import Dict, List, Tuple, Callable

import cv2
import numpy as np
import tensorflow as tf
from data import CelebADataset
from tensorflow.keras.callbacks import (
    Callback,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard,
)
from tensorflow.keras.layers import (
    BatchNormalization,
    Conv2D,
    Conv2DTranspose,
    Dropout,
    Flatten,
    Input,
    LeakyReLU,
    Reshape,
    Activation,
    Dense,
    Lambda,
    Concatenate
)
from tensorflow.keras import Sequential 
from tensorflow.keras.metrics import Mean
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Optimizer
from PIL import Image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from skimage.transform import resize
from numpy import cov, iscomplexobj, trace
from scipy.linalg import sqrtm
from keras import backend as K
from utils import SaveImagesCallback, FID
from keras.activations import tanh

hidden_size = 200
img_size = (128, 128, 3)

In [6]:
critic_input = Input(shape=img_size)
x = Conv2D(32, kernel_size=4, strides=2, padding="same")(critic_input)
x = LeakyReLU(0.2)(x)
x = Conv2D(32, kernel_size=4, strides=2, padding="same")(x)
x = LeakyReLU(0.2)(x)
x = Conv2D(32, kernel_size=4, strides=2, padding="same")(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)
x = Conv2D(32, kernel_size=4, strides=2, padding="same")(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.3)(x)
x = Conv2D(32, kernel_size=4, strides=2, padding="same")(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.3)(x)
x = Conv2D(1, kernel_size=4, strides=1, padding="valid")(x)
critic_output = Flatten()(x)

critic = Model(critic_input, critic_output, name="discriminator")

2024-05-23 19:58:25.552387: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22453 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:17:00.0, compute capability: 8.6
2024-05-23 19:58:25.552909: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5858 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [48]:
def sampling(args):
        mean_mu, log_var = args
        epsilon = K.random_normal(shape=K.shape(mean_mu), mean=0.0, stddev=1.0)
        return mean_mu + K.exp(log_var / 2) * epsilon
    
encoder_input = Input(shape=img_size, name='encoder-input')
conv1 = Sequential([
    Conv2D(64, kernel_size=4, strides=2, padding='same', use_bias=False),
    BatchNormalization(momentum=0.9), LeakyReLU(0.2)
], name='conv1')(encoder_input)
conv2 = Sequential([
    Conv2D(128, kernel_size=4, strides=2, padding='same', use_bias=False),
    BatchNormalization(momentum=0.9), LeakyReLU(0.2)
], name='conv2')(conv1)
conv3 = Sequential([
    Conv2D(256, kernel_size=4, strides=2, padding='same', use_bias=False),
    BatchNormalization(momentum=0.9), LeakyReLU(0.2)
], name='conv3')(conv2)
conv4 = Sequential([
    Conv2D(512, kernel_size=4, strides=2, padding='same', use_bias=False),
    BatchNormalization(momentum=0.9), LeakyReLU(0.2)
], name='conv4')(conv3)
conv5 = Sequential([
    Conv2D(512, kernel_size=4, strides=2, padding='same', use_bias=False),
    BatchNormalization(momentum=0.9), LeakyReLU(0.2)
], name='conv5')(conv4)
latent = Sequential([
    Conv2D(512, kernel_size=4, strides=2, padding='same', use_bias=False),
    BatchNormalization(momentum=0.9), LeakyReLU(0.2),
    Flatten()
], name='latent')(conv5)

mean = Dense(hidden_size, name='mean')(latent)
logvar = Dense(hidden_size, name='log-var')(latent)
encoder_output = Lambda(sampling, name="encoder-output")([mean, logvar])
encoder = Model(encoder_input, encoder_output, name='encoder')

print(encoder.summary())

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder-input (InputLayer)  [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv1 (Sequential)          (None, 64, 64, 64)           3328      ['encoder-input[0][0]']       
                                                                                                  
 conv2 (Sequential)          (None, 32, 32, 128)          131584    ['conv1[0][0]']               
                                                                                                  
 conv3 (Sequential)          (None, 16, 16, 256)          525312    ['conv2[0][0]']               
                                                                                            

In [59]:
decoder_input = Input(shape=(hidden_size, ), name='decoder-input')
deconv7 = Sequential([
    Reshape(target_shape=(1, 1, hidden_size)),
    Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False),
    BatchNormalization(momentum=0.9), LeakyReLU(0.2),
], name='deconv7')(decoder_input)
deconv6 = Sequential([
    Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False),
    BatchNormalization(momentum=0.9), LeakyReLU(0.2),
], name='deconv6')(deconv7)
deconv5 = Sequential([
    Concatenate(-1),
    Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False),
    BatchNormalization(momentum=0.9), LeakyReLU(0.2),
], name='deconv5')([deconv6, conv5])
# deconv4 = Sequential([
#     Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False),
#     BatchNormalization(momentum=0.9), LeakyReLU(0.2),
# ], name='deconv4')(deconv5)
# deconv3 = Sequential([
#     Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False),
#     BatchNormalization(momentum=0.9), LeakyReLU(0.2),
# ], name='deconv3')(deconv4)
# deconv2 = Sequential([
#     Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False),
#     BatchNormalization(momentum=0.9), LeakyReLU(0.2),
# ], name='deconv2')(deconv3)
# deconv1 = Sequential([
#     Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False),
#     BatchNormalization(momentum=0.9), LeakyReLU(0.2),
# ], name='deconv1')(deconv2)
# decoder_output = Sequential([
#     Conv2DTranspose(3, kernel_size=1, use_bias=False),
#     Activation(tanh)
# ], name='output')(deconv1)

decoder = Model(decoder_input, deconv5, name='decoder')
decoder.summary()

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='encoder-input'), name='encoder-input', description="created by layer 'encoder-input'") at layer "conv1". The following previous layers were accessed without issue: []